In [1]:
import pandas as pd
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

In [2]:
path_cru = "../data/CRU/"
path_grid = "../results/"
fname_cru = "mexico_cru_ts4.05.1901.2020.pre.dat.nc"
fname_grid = "grid_points.csv"

# Se leen los límites de la cuenca.
df_grid = pd.read_csv(path_grid + fname_grid)
lon_i = df_grid.lonp.min()
lon_f = df_grid.lonp.max() + 1
lat_i = df_grid.latp.min()
lat_f = df_grid.latp.max() + 1

# Se carga la precipitación.
pre_xr = xr.load_dataset(path_cru + fname_cru)
# Se recorta la zona de estudio.
pre_xr = pre_xr[dict(lon = slice(lon_i, lon_f), lat = slice(lat_i, lat_f))]
# Se promedia espacialmente y se retira stn, que no se requiere.
pre_df = pre_xr.mean(dim = "lat").mean(
    dim = "lon").to_dataframe().drop("stn", axis = 1)

In [3]:
pre_xr

<xarray.Dataset>
Dimensions:  (lat: 3, lon: 2, time: 1440)
Coordinates:
  * lat      (lat) float32 18.75 19.25 19.75
  * lon      (lon) float32 -99.25 -98.75
  * time     (time) datetime64[ns] 1901-01-16 1901-02-15 ... 2020-12-16
Data variables:
    pre      (time, lat, lon) float32 0.0 0.0 0.0 0.0 0.0 ... 1.7 1.4 1.2 1.3
    stn      (time, lat, lon) float64 2.0 2.0 2.0 2.0 2.0 ... 8.0 8.0 8.0 8.0
Attributes:
    Conventions:  CF-1.4
    title:        CRU TS4.05 Precipitation
    institution:  Data held at British Atmospheric Data Centre, RAL, UK.
    source:       Run ID = 2103051243. Data generated from:pre.2103041709.dtb
    history:      Thu May 13 20:11:00 2021: ncks -d lat,14.25,32.75 -d lon,-1...
    references:   Information on the data is available at http://badc.nerc.ac...
    comment:      Access to these data is available to any registered CEDA user.
    contact:      support@ceda.ac.uk
    NCO:          netCDF Operators version 4.8.1 (Homepage = http://nco.sf.ne...

In [4]:
pre_xr.close()

In [5]:
# Se seleccionan los datos a partir de 1979.
pre_df = pre_df[pre_df.index > "1979"]
# Se suman los valores para cada mes.
pre_df = pre_df.resample("M").sum()
# Se obtienen los nombres de los meses.
pre_df["Mes"] = pre_df.index.month_name("es_MX")
# Se cambia el formato del índice para que no incluya el día.
pre_df.index = pre_df.index.to_period("M")

In [17]:
# Se grafica el promedio mensual.
pre_mes = pre_df.groupby(pre_df.index.month).mean()
pre_mes.rename_axis("Mes", axis = "index", inplace = True)
pre_mes.plot.bar(legend = None)

# Se configura la gráfica.
plt.xlabel("Mes")
plt.ylabel("Precipitación [mm]")
plt.title(
    "Precipitación media en la\nCuenca del Valle de México",
    {'fontsize': 18}
    )
plt.xticks(range(0,12), pre_df.Mes[0:12], rotation = 60)

# Se muesta la gráfica.
plt.show()

ValueError: No axis named time for object type DataFrame

In [16]:
pre_mes

,pre
time,
1,8.873810
2,4.828175
3,8.670635
4,23.578571
5,73.112701
6,159.550003
7,171.698013
8,179.344849
9,168.662308


In [8]:
# Análisis de temporada de lluvias.
print("Precipitación total: " + str(pre_mes.sum()[0]))
print("Porcentaje de precipitación total de mayo a octubre: "
    + str(pre_mes.iloc[4:10].sum()[0] / pre_mes.sum()[0]))
print("Porcentaje de precipitación total de junio a septiembre: "
    + str(pre_mes.iloc[5:9].sum()[0] / pre_mes.sum()[0]))

Precipitación total: 893.33167
Porcentaje de precipitación total de mayo a octubre: 0.9198253
Porcentaje de precipitación total de junio a septiembre: 0.76036173
